# Wikipedia Indexing with ColBERT

In [1]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from rank_bm25 import BM25Okapi
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

In [2]:
pl.Config(fmt_str_lengths=2000);

In [3]:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

In [4]:
#!pip install rank_bm25

In [5]:
# !conda list | grep bitsandbytes

In [6]:
# !conda install -c conda-forge pyarrow -y

In [7]:
# !conda update -c conda-forge 'auto-gptq[triton]' -y

In [8]:
# huggingface_hub.login(os.environ['HUGGING_FACE_TOKEN'])

In [9]:
questions = pl.concat([
    pl.read_csv('./data/daniel_train_aug_19_200-300.csv'),
    pl.read_csv('./data/daniel_train_aug_19_400-500.csv')
])
questions.shape, questions.columns

((200, 10),
 ['question',
  'correct',
  'incorrect_1',
  'incorrect_2',
  'incorrect_3',
  'incorrect_4',
  'incorrect_5',
  'incorrect_6',
  'title',
  'section_title'])

In [10]:
wiki_sections = pl.read_parquet('./data/wiki_with_category.parquet')
wiki_sections.shape

(153622, 14)

In [19]:
def split_text_into_chunks(text, max_words=350):
    chunks = []
    current_chunk = []
    current_length = 0
    last_period = -1
    section_words = text.split()

    for word in section_words:
        if word.endswith('.'):
            last_period = current_length

        if current_length < max_words:
            current_length += 1
        else:
            if last_period > -1:
                cut_point = last_period + 1
                chunks.append(" ".join(section_words[:cut_point]))
                section_words = section_words[cut_point:]
                last_period = -1
                current_length = 0
            else:
                # If no period exists, just split it at max_words
                chunks.append(" ".join(section_words[:current_length]))
                section_words = section_words[current_length:]
                current_length = 0           

    if section_words:
        chunks.append(" ".join(section_words))
    
    return chunks

passages = wiki_sections[['section_id', 'title', 'section_title', 'section_text']]
passages = passages.with_columns(
    pl.col("section_text")
      .apply(split_text_into_chunks)\
      .cast(pl.List(pl.Utf8))\
)
passages = passages.explode('section_text')
passages = passages.filter(pl.col('section_text').is_not_null())\
                   .with_row_count('passage_id')

passages = passages.with_columns(pl.col('section_text').str.replace_all('\n', ' '))                      
passages_file = './data/wiki_passages'
passages[['passage_id', 'section_text']].write_csv(passages_file + '_colbert.tsv', separator='\t', has_header=False)
passages.write_parquet(passages_file+'.parquet')
passages.shape

(206109, 5)

In [12]:
passages.filter(pl.col('passage_id') == 138897)

passage_id,section_id,title,section_title,section_text
u32,u32,str,str,str
138897,100994,"""Special unitary group""","""Relation to spatial rotations""","""Main3D rotation group#Connection between SO(3) and SU(2)Quaternions and spatial rotation Every versor is naturally associated to a spatial rotation in 3 dimensions, and the product of versors is associated to the composition of the associated rotations. Furthermore, every rotation arises from exactly two versors in this fashion. In short: there is a 2:1 surjective homomorphism from SU(2) to 3D rotation group; consequently SO(3) is isomorphic to the quotient group SU(2)/{±I}, the manifold underlying SO(3) is obtained by identifying antipodal points of the 3-sphere S3 , and SU(2) is the Covering group#Universal covering group of SO(3)."""


In [13]:
wiki_sections.filter(pl.col('section_id') == 120418)[['title', 'section_title', 'section_text']]

title,section_title,section_text
str,str,str
"""Hylomorphism (computer science)""","""Hylomorphisms in practice""","""Lists List (computing) are common data structures as they naturally reflect linear computational processes. These processes arise in repeated (Iteration) function calls. Therefore, it is sometimes necessary to generate a temporary list of intermediate results before reducing this list to a single result. One example of a commonly encountered hylomorphism is the canonical factorial function. syntaxhighlightlanghaskell factorial :: Integer -> Integer factorial n | n == 0 = 1 | n > 0 = n * factorial (n - 1) syntaxhighlight In the previous example (written in Haskell (programming language), a purely functional programming language) it can be seen that this function, applied to any given valid input, will generate a linear call tree isomorphism to a list. For example, given n = 5 it will produce the following: factorial 5 = 5 * (factorial 4) = 120 factorial 4 = 4 * (factorial 3) = 24 factorial 3 = 3 * (factorial 2) = 6 factorial 2 = 2 * (factorial 1) = 2 factorial 1 = 1 * (factorial 0) = 1 factorial 0 = 1 In this example, the anamorphic part of the process is the generation of the call tree which is isomorphic to the list code[1, 1, 2, 3, 4, 5]code. The catamorphism, then, is the calculation of the product (mathematics) of the element (mathematics) of this list. Thus, in the notation given above, the factorial function may be written as textfactorial = [![(1,times),(g, p)]!] where g n = (n, n - 1) and p n = (n = 0). Trees However, the term 'hylomorphism' does not apply solely to functions acting upon isomorphisms of lists. For example, a hylomorphism may also be defined by generating a non-linear call tree which is then collapsed. An example of such a function is the function to generate the nth Term (logic) of the Fibonacci sequence. syntaxhighlightlanghaskell fibonacci :: Integer -> Integer fibonacci n | n == 0 = 0 | n == 1 = 1 | n > 1 = fibonacci (n - 2) + fibonacci (n - 1) syntaxhighlight Image:Fib 4.png This function, again applied …"


In [15]:
tokenized_corpus = [doc.split(" ") for doc in passages['section_text']]
bm25_corpus = BM25Okapi(tokenized_corpus)

In [16]:
def bm25_scores(query):
    tokenized_query = query.split(" ")
    scores = pd.Series(bm25_corpus.get_scores(tokenized_query))
    scores = scores.sort_values(ascending=False)
    return scores

scores = []
n_questions = 20
n_results_per_question = 2
for question in tqdm(questions['question'][:n_questions]):
    q_scores = pd.Series(bm25_scores(question))
    top_10 = q_scores.sort_values(ascending=False)[:n_results_per_question].index.to_list()
    scores.append(top_10)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# pl.Series("bm25", scores, dtype=pl.List(pl.UInt32))

In [17]:
bm25_res = pl.Series("bm25", scores, dtype=pl.List(pl.UInt32))
bm25_res = questions[:n_questions][['question', 'title', 'section_title']].with_columns(bm25_res)
bm25_res = bm25_res.with_row_count('qid')
bm25_res = bm25_res.explode('bm25')
bm25_res = bm25_res.with_columns(pl.lit(1).alias("ones"))\
                   .select([
                       pl.all().exclude("ones"),
                       pl.col("ones").cumsum().over("qid").flatten().alias("result_idx")
                   ])
bm25_res = bm25_res.join(passages, how='left', left_on='bm25', right_on='passage_id', suffix='_bm25', validate='m:1')
# bm25_res.filter(pl.col('result_idx') < 4)

## Using ColBERT to find Wiki Section

In [18]:
queries = questions.with_row_count('qid')[['qid', 'question']]
queries = queries.with_columns(pl.col('question').str.replace_all('\n', ' ')) 
queries_file = './data/wiki_queries.tsv'
queries.write_csv(queries_file, separator='\t', has_header=False)

In [33]:
c_collection = Collection(passages_file)
c_queries = Queries(queries_file)
f'Loaded {len(c_queries)} queries and {len(c_collection):,} passages'

[Aug 29, 13:25:41] #> Loading collection...
0M 
[Aug 29, 13:25:41] #> Loading the queries from ./data/wiki_queries.tsv ...
[Aug 29, 13:25:41] #> Got 200 queries. All QIDs are unique.



'Loaded 200 queries and 206,109 passages'

In [34]:
ColBERTConfig?

Init signature:
ColBERTConfig(
    query_token_id: str = DefaultVal(val='[unused0]'),
    doc_token_id: str = DefaultVal(val='[unused1]'),
    query_token: str = DefaultVal(val='[Q]'),
    doc_token: str = DefaultVal(val='[D]'),
    ncells: int = DefaultVal(val=None),
    centroid_score_threshold: float = DefaultVal(val=None),
    ndocs: int = DefaultVal(val=None),
    index_path: str = DefaultVal(val=None),
    nbits: int = DefaultVal(val=1),
    kmeans_niters: int = DefaultVal(val=4),
    resume: bool = DefaultVal(val=False),
    similarity: str = DefaultVal(val='cosine'),
    bsize: int = DefaultVal(val=32),
    accumsteps: int = DefaultVal(val=1),
    lr: float = DefaultVal(val=3e-06),
    maxsteps: int = DefaultVal(val=500000),
    save_every: int = DefaultVal(val=None),
    warmup: int = DefaultVal(val=None),
    warmup_bert: int = DefaultVal(val=None),
    relu: bool = DefaultVal(val=False),
    nway: int = DefaultVal(val=2),
    use_ib_negatives: bool = DefaultVal(val=False),
 

In [35]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 512   # truncate passages at 300 tokens
dim = 128 # default is 128

checkpoint = './checkpoints/open-nq-colbert-xlmr-large'
experiment = 'wiki-science'
indexer_name = f"wiki_pages_index_{nbits}bits"

os.environ['COLBERT_LOAD_TORCH_EXTENSION_VERBOSE'] = 'True'

config = ColBERTConfig(
    doc_maxlen=doc_maxlen,
    nbits=nbits,
    dim=dim
)

In [36]:
rerun = False
if rerun:
    with Run().context(RunConfig(nranks=1, experiment=experiment)):
        indexer = Indexer(checkpoint=checkpoint, config=config)
        indexer.index(name=indexer_name,
                      collection=c_collection,
                      overwrite=True)

In [37]:
with Run().context(RunConfig(nranks=1, experiment=experiment)):
    searcher = Searcher(index=indexer_name, config=config)
    ranking = searcher.search_all(c_queries, k=n_results_per_question)

[Aug 29, 13:25:41] #> Loading collection...
0M 
[Aug 29, 13:25:42] #> Loading codec...
[Aug 29, 13:25:42] #> Loading IVF...
[Aug 29, 13:25:42] #> Loading doclens...


100%|██████████| 9/9 [00:00<00:00, 2234.84it/s]

[Aug 29, 13:25:42] #> Loading codes and residuals...



100%|██████████| 200/200 [00:01<00:00, 162.67it/s]


In [38]:
# info.with_columns(pl.lit(1).alias("ones"))\
#     .select([
#         pl.all().exclude("ones"),
#         pl.col("ones").cumsum().over("qid").flatten().alias("bm25_idx")
#     ])

In [39]:
colbert = [tup[1] for tup in ranking.flat_ranking]
colbert = pl.Series("colbert", colbert[:n_questions * n_results_per_question], dtype=pl.UInt32)
res = bm25_res.with_columns(colbert)
res = res.join(passages, how='left', left_on='colbert', right_on='passage_id', validate='m:1', suffix='_col')
res = res.with_columns([
    (pl.col('title') == pl.col('title_bm25')).alias('bm25_title_match'),
    (pl.col('title') == pl.col('title_col')).alias('col_title_match')
])

In [40]:
res.filter(pl.col('result_idx') == 1)['bm25_title_match'].mean()

0.515

In [41]:
res.filter(pl.col('result_idx') == 1)['col_title_match'].mean()

0.825

### Recall at 10

In [111]:
res.filter(pl.col('result_idx') <= 10).groupby('qid').agg(pl.max('bm25_title_match'))['bm25_title_match'].mean()

0.745

In [121]:
res.filter(pl.col('result_idx') <= 5).groupby('qid').agg(pl.max('col_title_match'))['col_title_match'].mean()

0.97

In [122]:
question = 2
res[['question', 'section_text', 'section_text_col']][question*n_results_per_question:(question+1)*n_results_per_question]

question,section_text,section_text_col
str,str,str
"""What does Bochner's theorem in mathematics characterize?""","""Use American Englishdate March 2019 Short descriptionTheorem of Fourier transforms of Borel measures AboutBochner's theorem in harmonic analysisBochner's theorem in Riemannian geometryBochner's theorem (Riemannian geometry) In mathematics, Bochner's theorem (named for Salomon Bochner) characterizes the Fourier transform of a positive finite Borel measure on the real line. More generally in harmonic analysis, Bochner's theorem asserts that under Fourier transform a continuous Positive-definite function on a group on a locally compact group abelian group corresponds to a finite positive measure on the Pontryagin duality. The case of sequences was first established by Gustav Herglotz (see also the related Herglotz representation theorem.)""","""Use American Englishdate March 2019 Short descriptionTheorem of Fourier transforms of Borel measures AboutBochner's theorem in harmonic analysisBochner's theorem in Riemannian geometryBochner's theorem (Riemannian geometry) In mathematics, Bochner's theorem (named for Salomon Bochner) characterizes the Fourier transform of a positive finite Borel measure on the real line. More generally in harmonic analysis, Bochner's theorem asserts that under Fourier transform a continuous Positive-definite function on a group on a locally compact group abelian group corresponds to a finite positive measure on the Pontryagin duality. The case of sequences was first established by Gustav Herglotz (see also the related Herglotz representation theorem.)"""
"""What does Bochner's theorem in mathematics characterize?""","""Then u* A(f*) u sum_j, k 1n overlineu_k u_j f*(x_k - x_j) sum_j, k 1n overlineu_k u_j f(phi(x_k) - phi(x_j)) = u* tildeA(f) u ge 0, where tildeA(f) = big( f(phi(x_i) - phi(x_j)) = f(tildex_i - tildex_j) big)_i, j where tildex_k := phi(x_k) are distinct as phi is linear. Bochner's theorem mainBochner's theorem Positive-definiteness arises naturally in the theory of the Fourier transform; it can be seen directly that to be positive-definite it is sufficient for f to be the Fourier transform of a function g on the real line with g(y) ≥ 0. The converse result is Bochner's theorem, stating that any continuous function positive-definite function on the real line is the Fourier transform of a (positive) measure (mathematics). Applications In statistics, and especially Bayesian statistics, the theorem is usually applied to real functions. Typically, n scalar measurements of some scalar value at points in Rd are taken and points that are mutually close are required to have measurements that are highly correlated. In practice, one must be careful to ensure that the resulting covariance matrix (an nowrapn×n matrix) is always positive-definite. One strategy is to define a correlation matrix A which is then multiplied by a scalar to give a covariance matrix: this must be positive-definite. Bochner's theorem states that if the correlation between two points is dependent only upon the distance between them (via function f), then function f must be positive-definite to ensure the covariance matrix A is positive-definite. See Kriging. In this context, Fourier terminology is not normally used and instead it is stated that f(x) is the characteristic function (probability theory) of a symmetric probability density function. Generalization mainPositive-definite function on a group One can define positive-definite functions on any locally compact abelian topological group; Bochner's theorem extends to this context. Positive-definite functions on groups occur naturally in the representat…","""Bochner's theorem in the special case of the discrete group Z is often referred to as Herglotz's theorem (see Herglotz representation theorem) and says that a function f on Z with f(0) = 1 is positive-definite if and only if there exists a probability measure μ on the circle T such that f(k) = int_mathbbT e-2 

In [56]:
res = res.with_columns([
    (pl.col('title') == pl.col('title_bm25')).alias('bm25_title_match'),
    (pl.col('title') == pl.col('title_col')).alias('col_title_match')
])

qid,question,title,section_title,bm25,result_idx,section_id,title_bm25,section_title_bm25,section_text,colbert,section_id_col,title_col,section_title_col,section_text_col,bm25_title_match,col_title_match
u32,str,str,str,u32,i32,u32,str,str,str,u32,u32,str,str,str,bool,bool
0,"""In mathematics, what is an iterated function?""","""Iterated function""","""Fixed points""",164378,1,120939,"""Iterated function""","""Summary""","""Use dmy datesdateMay 2019cs1-datesy File:An iterated direct similarity yields spirals.svg In mathematics, an iterated function is a function X→X (that is, a function from some Set (mathematics) mvarX to itself) which is obtained by function composition another function f:X→X with itself a certain number of times. The process of repeatedly applying the same function is called iteration. In this process, starting from some initial object, the result of applying a given function is fed again in the function as input, and this process is repeated. For example on the image on the right: nobr1L = mathitF,( K ), M = mathitF,circ mathitF,( K ) = mathitF;2,( K ),brwith the circle‑shaped symbol of function composition. Iterated functions are objects of study in computer science, fractals, dynamical systems, mathematics and renormalization group physics.""",164394,120954,"""Iterated function""","""In computer science""","""In computer science, iterated functions occur as a special case of recursion (computer science), which in turn anchor the study of such broad topics as lambda calculus, or narrower ones, such as the denotational semantics of computer programs.""",true,true
0,"""In mathematics, what is an iterated function?""","""Iterated function""","""Fixed points""",170412,2,125672,"""Lefschetz zeta function""","""Summary""","""In mathematics, the Lefschetz zeta function is a tool used in topological periodic and fixed point (mathematics) theory, and dynamical systems. Given a continuous map fcolon Xto X, the zeta-function is defined as the formal series zeta_f(t) = exp left( sum_n=1infty L(fn) fractnn right), where L(fn) is the Lefschetz number of the n-th iterated function of f. This zeta-function is of note in topological periodic point theory because it is a single invariant containing information about all iterates of f.""",164378,120939,"""Iterated function""","""Summary""","""Use dmy datesdateMay 2019cs1-datesy File:An iterated direct similarity yields spirals.svg In mathematics, an iterated function is a function X→X (that is, a function from some Set (mathematics) mvarX to itself) which is obtained by function composition another function f:X→X with itself a certain number of times. The process of repeatedly applying the same function is called iteration. In this process, starting from some initial object, the result of applying a given function is fed again in the function as input, and this process is repeated. For example on the image on the right: nobr1L = mathitF,( K ), M = mathitF,circ mathitF,( K ) = mathitF;2,( K ),brwith the circle‑shaped symbol of function composition. Iterated functions are objects of study in computer science, fractals, dynamical systems, mathematics and renormalization group physics.""",false,true
0,"""In mathematics, what is an iterated function?""","""Iterated function""","""Fixed points""",177874,3,131625,"""Expansive homeomorphism""","""Summary""","""Aboutmathematical expansivityother usesexpansivity (disambiguation)andexpansion (disambiguation)andExpanse (disambiguation) In mathematics, the notion of expansivity formalizes the notion of points moving away from one another under the action of an iterated function. The idea of expansivity is fairly rigidity (mathematics), as the definition of positive expansivity, below, as well as the Schwarz–Ahlfors–Pick theorem demonstrate.""",177874,131625,"""Expansive homeomorphism""","""Summary""","""Aboutmathematical expansivityother usesexpansivity (disambiguation)andexpansion (disambiguation)andExpanse (disambiguation) In mathemat